In [ ]:
# delete this cell if working on Pycharm
!pip install Bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from Bio.PDB import *
import numpy as np
import os
from tqdm import tqdm

In [ ]:
## constants:

AA_DICT = {"A": 0, "C": 1, "D": 2, "E": 3, "F": 4, "G": 5, "H": 6, "I": 7, "K": 8, "L": 9, "M": 10, "N": 11,
           "P": 12, "Q": 13, "R": 14, "S": 15, "T": 16, "W": 17, "Y": 18, "V": 19, "X": 20, "-": 21}
UNIQE_AA = {"UNK": "X", "TYS": "Y", "FME": "M", "PCA": "Q", "CSD":"C", "MLY":"K", "SEP":"S", "YCM":"C", "CSX": "C", "NEP":"H", "IAS":"D", "PM3": "X"}
FEATURE_NUM = len(AA_DICT) + 2 
BACKBONE_ATOMS = ["N", "CA", "C", "O", "CB"]
OUTPUT_SIZE = len(BACKBONE_ATOMS) * 3

SH2_MAX_LEN = 140 + 10 # maximum length in data + safety range
PEP_MAX_LEN = 25 + 10 # maximum length in data + safety range

In [ ]:
def get_seq_aa(pdb_file, chain_id):
    """
    returns the sequence (String) and a list of all the aa residue objects of the given protein chain.
    :param pdb_file: path to a pdb file
    :param chain_id: chain letter (char)
    :return: sequence, [aa objects]
    """
    # load model
    chain = PDBParser(QUIET=True).get_structure(pdb_file, pdb_file)[0][chain_id]

    aa_residues = []
    seq = ""
    for residue in chain:
        aa = residue.get_resname()
        if not is_aa(aa) or not residue.has_id('CA'):
            continue
        # elif aa in ["PM3", "PTH"]:
          # continue
        elif aa in UNIQE_AA:
            seq += UNIQE_AA[aa]
        else:
          try:
            seq += Polypeptide.three_to_one(residue.get_resname())
          except:
            seq += "X"
        aa_residues.append(residue)
    return seq, aa_residues

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def generate_single_input(pdb_file, chain_id, max_len): 
    """
    receives a pdb file and returns its sequence in a one-hot encoding matrix (each row is an aa in the sequence, and
    each column represents a different aa out of the 20 aa + 2 special columns).
    :param pdb_file: path to a pdb file
    :param chain_id: chain ID of the chain we want to genarate label to
    :param max_len: maximum possible length of input (rows)
    :return: numpy array of shape (max_len, FEATURE_NUM)
    """

    # get seq and aa residues
    seq, _ = get_seq_aa(pdb_file, chain_id)

    output_map = np.zeros((max_len,FEATURE_NUM))
    seq = seq + ((max_len - len(seq)) * "-")
    for i in range(max_len):
        output_map[i, AA_DICT[seq[i]]] = 1
    return output_map
    

In [ ]:
def generate_input(sh2_path, sh2_chain_id, pep_path, pep_chain_id):
  """
  receives 2 pdbs files and returns both of them sequence in a one-hot encoding 
  matrix (each row is an aa in the sequence, and
  each column represents a different aa out of the 20 aa + 2 special columns).
  in addition, last 2 colunms separates between sh2 and peptide
  :param sh2_path: path to a pdb file of sh2
  :param sh2_chain_id: chain ID of the SH2 we want to genarate label to
  :param pep_path: path to a pdb file of a peptide
  :param pep_chain_id: chain ID of the peptie we want to genarate label to
  :return: numpy array of shape (SH2_MAX_LEN+PEP_MAX_LEN, FEATURE_NUM)
  """
  sh2_input = generate_single_input(sh2_path, sh2_chain_id, SH2_MAX_LEN)
  pep_input = generate_single_input(pep_path, pep_chain_id, PEP_MAX_LEN,)
  
  # last 2 colunms separates between sh2 and peptide
  sh2_input[:,22] = 1
  pep_input[:,23] = 1
  return np.concatenate([sh2_input, pep_input], axis=0)


In [ ]:
def generate_single_label(pdb_file, chain_id, max_len):  # TODO: implement this!

    """
    receives a pdb file and returns its pairwise distances and pairwise angles (omega, theta, phi).
    :param pdb_file: path to a pdb file (nanobody, heavy chain has id 'H')
    :return: 4 numpy arrays, the first one with size (CDR_MAX_LENGTH * CDR_MAX_LENGTH * 1) and the other three with
    size (max_len * max_len * 2).
    """
    # get seq and aa residues
    seq, aa_residues = get_seq_aa(pdb_file, chain_id)

    # turn into backbone + CB xyz matrix
    xyz_matrix = np.zeros((max_len, OUTPUT_SIZE))
    for i in range(len(aa_residues)):
        for j, atom in enumerate(BACKBONE_ATOMS):
            if not (atom=="CB" and seq[i] == "G"):
                if atom in [a.id for a in aa_residues[i].get_atoms()]:
                    xyz_matrix[i][3*j:3*j+3] = aa_residues[i][atom].get_coord()

    return xyz_matrix

In [ ]:
def generate_label(sh2_path, sh2_chain_id, pep_path, pep_chain_id):
  """
  receives 2 pdb files and returns both of thembackbone + CB coordinates.
  :param sh2_path: path to a pdb file of sh2
  :param sh2_chain_id: chain ID of the SH2 we want to genarate label to
  :param pep_path: path to a pdb file of a peptide
  :param pep_chain_id: chain ID of the peptie we want to genarate label to
  :return: numpy array of shape (SH2_MAX_LEN+PEP_MAX_LEN, OUTPUT_SIZE).
  """
  sh2_label = generate_single_label(sh2_path, sh2_chain_id, SH2_MAX_LEN)
  pep_label = generate_single_label(pep_path, pep_chain_id, PEP_MAX_LEN)
  return np.concatenate([sh2_label, pep_label], axis=0)

In [ ]:
if __name__ == '__main__':
    
    # this scripts creates input and labels to SH2 and peptide neural network,
    # from data in data path
    
    # data path is a path to a directory:
    # in the directory there are several directories, representing different pdbs, in each we have:
    # pdb-name_SH2_<chain_id>.pdb
    # pdb-name_peptide_<chain_id>.pdb

    input_matrix = []
    labels_matrix = []

    all_seqs_sh2 = []
    all_seqs_pep = []

    all_pdbs = []

    empty_files = 0

    # TODO: change path to your data
    data_path = "/content/drive/MyDrive/protein_hackaton_data/actual_data" 
    
    for pdb_dir in tqdm(os.listdir(data_path)): # iterate all dirs
      empty_f = False
      print("now scanning dir: ", pdb_dir)
      had_sh2 = False
      has_pep = False
      pdb_dir = os.path.join(data_path, pdb_dir)
      for pdb in tqdm(os.listdir(pdb_dir)): # iterate sh2, pep
        assert pdb.endswith(".pdb"), f"invalid file name {pdb}"
        pdb_no_suffix = pdb[:-4] # remove .pdb
        pdb_l = pdb_no_suffix.split("_")
        assert len(pdb_l)==3, f"invalid file name {pdb}"
        pdb_name, protein_type, chain_id = pdb_l
        if protein_type == "SH2":
          sh2_path = os.path.join(pdb_dir, pdb)
          if os.path. getsize(sh2_path) == 0:
            raise Exception(f'File {sh2_path} is empty')
          sh2_chain_id = chain_id
          had_sh2 = True
        elif protein_type == "peptide":
          pep_path = os.path.join(pdb_dir, pdb)
          if os.path. getsize(pep_path) == 0:
            raise Exception(f'File {pep_path} is empty')
          pep_chain_id = chain_id
          has_pep = True
        else:
          raise Exception(f"invalid protein type {protein_type}")

      if empty_f:
        continue
      if not had_sh2 or not has_pep:
        raise Exception("sh2 or pep are missing!")

      input = generate_input(sh2_path, sh2_chain_id, pep_path, pep_chain_id)
      input_matrix.append(input)


      label = generate_label(sh2_path, sh2_chain_id, pep_path, pep_chain_id)
      labels_matrix.append(label)

      seq_sh2, _ = get_seq_aa(sh2_path, sh2_chain_id)
      seq_pep, _ = get_seq_aa(pep_path, pep_chain_id)

      all_seqs_sh2.append(seq_sh2)
      all_seqs_pep.append(seq_pep)

      all_pdbs.append(pdb_name)

    # TODO: change path to where you want to save the data you need for the network train and validation
    save_path = "/content/drive/MyDrive/protein_hackaton_data/data_to_network/"

    assert len(all_seqs_sh2) == len(input_matrix)
    
    np.save(f"{save_path}/train_input.npy", np.array(input_matrix))
    np.save(f"{save_path}/train_labels.npy", np.array(labels_matrix))
    
    with open(f"{save_path}/all_seqs_sh2.txt", "w") as f:
      for s in all_seqs_sh2:
        f.write(str(s) +"\n")
    
    with open(f"{save_path}/all_seqs_pep.txt", "w") as f:
      for s in all_seqs_pep:
          f.write(str(s) +"\n")

    with open(f"{save_path}/all_pdbs.txt", "w") as f:
      for s in all_pdbs:
          f.write(str(s) +"\n")

    print(f"Number of samples: {len(input_matrix)}")


  0%|          | 0/91 [00:00<?, ?it/s]

now scanning dir:  pdb6roz



100%|██████████| 2/2 [00:00<00:00, 1886.78it/s]


now scanning dir:  pdb6roy



  2%|▏         | 2/91 [00:00<00:08, 10.85it/s]

...

now scanning dir:  pdb1csy



 99%|█████████▉| 90/91 [00:10<00:00,  5.89it/s]

now scanning dir:  pdb1aot



100%|██████████| 91/91 [00:10<00:00,  8.42it/s]

Number of samples: 91
